<a href="https://colab.research.google.com/github/gopichandchalla16/infosys-internship-real-time-industry-insight-system/blob/main/Realtime_Industry_Insight_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install wikipedia --quiet

In [ ]:
import openai, tweepy, pandas as pd, matplotlib.pyplot as plt

In [ ]:
print("✅ Environment ready!")

✅ Environment ready!


In [1]:
# Testing completed

Testing completed on colab


Infosys Project : Real-Time Industry Insight & Strategic Intelligence System

Sprint 1 – Data Sourcing & Handling


Objective :: To collect real-time data from external APIs, preprocess it, perform sentiment analysis using a pre-trained Hugging Face model, and visualize the results through a simple interactive dashboard.

 Step 1 :: Install Dependencies

In [2]:
!pip install wikipedia yfinance feedparser textblob transformers matplotlib --quiet


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.1 MB/s eta 0:00:00


⚙️ Step 1 – Imports

In [20]:
import pandas as pd
import wikipedia
import yfinance as yf
import feedparser
import re
from textblob import TextBlob
import matplotlib.pyplot as plt
from transformers import pipeline


In [4]:
import plotly.express as px
import plotly.graph_objects as go

💬 Step 2 – Get User Input

In [23]:
company = input("Enter the company name: ").strip()
print(f"\n📊 Analyzing data for: {company}\n")

Enter the company name: netflix

📊 Analyzing data for: netflix



Fetch Summary from Wikipedia

In [28]:
try:
    # fetch the first 3 sentences of the summary
    summary = wikipedia.summary(company, sentences=3)
    print(f"\n📘 Wikipedia Summary for {company}:\n")
    print(summary)
except wikipedia.DisambiguationError as e:
    print("⚠️ The name is ambiguous. Try one of these options:")
    print(e.options[:10])
except wikipedia.PageError:
    print("❌ No page found for that company name.")
except Exception as err:
    print("⚠️ Unexpected error:", err)


# extra feature

try:
    page = wikipedia.page(company)
    print("\n🔗 URL:", page.url)
    print("\n📑 Title:", page.title)
    print("\n📝 First 500 characters:\n")
    print(page.content[:500])
except Exception as err:
    print("⚠️ Could not fetch full page:", err)




📘 Wikipedia Summary for netflix:

Netflix is an American subscription video on-demand over-the-top streaming service. The service primarily distributes original and acquired films and television shows from various genres, and it is available internationally in multiple languages.
Launched in 2007, nearly a decade after Netflix, Inc.

🔗 URL: https://en.wikipedia.org/wiki/Netflix

📑 Title: Netflix

📝 First 500 characters:

Netflix is an American subscription video on-demand over-the-top streaming service. The service primarily distributes original and acquired films and television shows from various genres, and it is available internationally in multiple languages.
Launched in 2007, nearly a decade after Netflix, Inc. began its pioneering DVD-by-mail movie rental service, Netflix is the most-subscribed video on demand streaming media service, with 301.6 million paid memberships in more than 190 countries as of 202


🌐 Step 3 – Collect Real-Time Data

In [17]:
# Wikipedia
try:
    summary = wikipedia.summary(company, sentences=3)
except:
    summary = "No Wikipedia summary found."

# Yahoo Finance
try:
    ticker = yf.Ticker(company)
    market_data = ticker.history(period="1mo")[["Close"]]
    if market_data.empty:
        raise ValueError
except:
    print("⚠️ Could not fetch Yahoo Finance data; using TSLA as fallback.")
    market_data = yf.Ticker("TSLA").history(period="1mo")[["Close"]]

# Google News RSS
feed = feedparser.parse(f"https://news.google.com/rss/search?q={company}")
headlines = [entry.title for entry in feed.entries]

# Synthetic tweets
tweets = [f"{company} is doing great! Love the innovation #{i}" for i in range(25)] + \
         [f"{company} is facing issues lately #{i}" for i in range(25)]

print("✅ Data collected successfully.")


ERROR:yfinance:$INFOSYS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")


⚠️ Could not fetch Yahoo Finance data; using TSLA as fallback.
✅ Data collected successfully.


🧹 Step 4 – Pre-Process

In [8]:
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^A-Za-z\s]", "", text)
    return text.lower().strip()

data = {
    "Source": ["Wikipedia"] + ["News"] * len(headlines) + ["Tweet"] * len(tweets),
    "Text": [summary] + headlines + tweets
}
df = pd.DataFrame(data)
df["Cleaned_Text"] = df["Text"].apply(clean_text)


🤖 Step 5 – Sentiment Analysis (TextBlob for speed)

In [9]:
df["Polarity"] = df["Cleaned_Text"].apply(lambda x: TextBlob(x).sentiment.polarity)
df["Sentiment"] = df["Polarity"].apply(
    lambda p: "Positive" if p > 0 else "Negative" if p < 0 else "Neutral"
)


📊 Step 6 – Interactive Visualizations (Plotly)

① Sentiment Distribution

In [10]:
sent_counts = df["Sentiment"].value_counts().reset_index()
sent_counts.columns = ["Sentiment", "Count"]

fig1 = px.bar(
    sent_counts,
    x="Sentiment", y="Count", color="Sentiment",
    title="Sentiment Distribution", text="Count",
    color_discrete_map={"Positive":"green","Negative":"red","Neutral":"gray"}
)
fig1.update_layout(xaxis_title="Sentiment Type", yaxis_title="Number of Mentions")
fig1.show()


② Market Price Trend

In [11]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=market_data.index, y=market_data["Close"],
    mode="lines+markers", name="Market Price",
    line=dict(width=2)
))
fig2.update_layout(
    title=f"{company} Market Price Trend (Last 30 Days)",
    xaxis_title="Date", yaxis_title="Price (USD)", hovermode="x"
)
fig2.show()

③ Tweet Sentiment Trend

In [12]:
tweets_df = df[df["Source"]=="Tweet"].copy()
tweets_df["Index"] = range(1, len(tweets_df)+1)

fig3 = px.line(
    tweets_df, x="Index", y="Polarity",
    title="Sentiment Trend Across Tweets",
    markers=True, line_shape="spline"
)
fig3.update_layout(xaxis_title="Tweet Index", yaxis_title="Sentiment Polarity (-1 to 1)")
fig3.show()

④ Sentiment by Source (Extra Insight)

In [13]:
fig4 = px.box(
    df, x="Source", y="Polarity", color="Source",
    title="Sentiment Distribution by Data Source"
)
fig4.update_traces(boxmean=True)
fig4.show()

📈 Step 7 – Summary Output

In [14]:
print("📊 Company Analyzed :", company)
print("📰 Wikipedia Summary :", summary)
print(f"😊 Average Sentiment Score : {df['Polarity'].mean():.3f}")
print(f"✅ Positive Tweets     : {(df['Sentiment']=='Positive').mean()*100:.1f}%")
print(f"❌ Negative Tweets     : {(df['Sentiment']=='Negative').mean()*100:.1f}%")


📊 Company Analyzed : infosys
📰 Wikipedia Summary : No Wikipedia summary found.
😊 Average Sentiment Score : 0.106
✅ Positive Tweets     : 36.4%
❌ Negative Tweets     : 22.5%


Alpha Vantage testing


Alpha Vantage is a robust API for financial data.

In [29]:
!pip install alpha_vantage --quiet


In [35]:
!pip install yfinance --quiet

Import neccessary file and input taken

USER INPUT


Fetch time‐series data from the alpha vantage